In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import dask.array as da
import pyproj
import sys
from joblib import Parallel, delayed
from tqdm import tqdm

pd.set_option("display.max_rows", 200)
import pdb

from pycontrails import Flight, Fleet, MetDataset
from pycontrails.core import models
from pycontrails.datalib.ecmwf import ERA5
from pycontrails.physics import geo, thermo, units, constants

# from pycontrails.models.ps_model import PSFlight
# from pycontrails.models.emissions import Emissions
from pycontrails.ext.flight_gen import FlightGen
from pycontrails.models.boxmodel.boxm_ac import Boxm
import pycontrails.models.boxmodel.boxm_script as boxm_script

# from pycontrails.models.dry_advection import DryAdvection
from pycontrails.core.met_var import (
    AirTemperature,
    RelativeHumidity,
    SpecificHumidity,
    EastwardWind,
    NorthwardWind,
    VerticalVelocity,
)

In [2]:
# meteorological parameters
met_params = {
    "air_temperature": 235.0,  # K
    "specific_humidity": 0.003,  # 1
    "relative_humidity": 0.5,  # 1
    "eastward_wind": 0.0,  # m/s
    "northward_wind": 0.0,  # m/s
    "lagrangian_tendency_of_air_pressure": 0.0,  # m/s
}

In [3]:
# flight trajectory parameters
fl_params = {
    "t0_fl": pd.to_datetime("2022-01-20 13:00:00"),  # flight start time
    "rt_fl": pd.Timedelta(minutes=60),  # flight run time
    "ts_fl": pd.Timedelta(minutes=2),  # flight time step
    "ac_type": "A320",  # aircraft type
    "fl0_speed": 100.0,  # m/s
    "fl0_heading": 0.0,  # deg
    "fl0_coords0": (0.1, 0.125, 12500),  # lat, lon, alt [deg, deg, m]
    "sep_dist": (5000, 2000, 0),  # dx, dy, dz [m]
    "n_ac": 1,  # number of aircraft
}

In [4]:
 # plume dispersion parameters
plume_params = {
    "dt_integration": pd.Timedelta(minutes=2),  # integration time step
    "max_age": pd.Timedelta(hours=2),  # maximum age of the plume
    "depth": 50.0,  # initial plume depth, [m]
    "width": 50.0,  # initial plume width, [m]
    "shear": 0.01,  # wind shear [1/s]
}

In [5]:
# chemistry sim parameters
chem_params = {
    "t0_chem": pd.to_datetime("2022-01-20 12:00:00"),  # chemistry start time
    "rt_chem": pd.Timedelta(hours=3),  # chemistry runtime
    "ts_chem": pd.Timedelta(seconds=20),  # chemistry time step
    "lat_bounds": (0.0, 1.0),  # lat bounds [deg]
    "lon_bounds": (0.0, 1.0),  # lon bounds [deg]
    "alt_bounds": (12000, 13000),  # alt bounds [m]
    "hres_pl": 0.01,  # horizontal resolution of the plume, [deg]
    "hres_chem": 0.01,  # horizontal resolution [deg]
    "vres_chem": 500,  # vertical resolution [m]
}

In [6]:
lats_pl = np.arange(
    chem_params["lat_bounds"][0], chem_params["lat_bounds"][1] + chem_params["hres_pl"], chem_params["hres_pl"]
)

lons_pl = np.arange(
    chem_params["lon_bounds"][0], chem_params["lon_bounds"][1] + chem_params["hres_pl"], chem_params["hres_pl"]
)

In [ ]:
lats = np.arange(
    chem_params["lat_bounds"][0], chem_params["lat_bounds"][1] + chem_params["hres_chem"], chem_params["hres_chem"]
)

lons = np.arange(
    chem_params["lon_bounds"][0], chem_params["lon_bounds"][1] + chem_params["hres_chem"], chem_params["hres_chem"]
)

alts = np.arange(
    chem_params["alt_bounds"][0], chem_params["alt_bounds"][1] + chem_params["vres_chem"], chem_params["vres_chem"]
)

times = pd.date_range(
    start=chem_params["t0_chem"],
    end=chem_params["t0_chem"] + chem_params["rt_chem"],
    freq=chem_params["ts_chem"],
)

In [ ]:
fl_df, pl_df, boxm = boxm_script.boxm_run(lons, lons_pl, lats, lats_pl, alts, times, fl_params, plume_params, chem_params, met_params)

In [ ]:
# ts = 6

# fl_df.loc[fl_df["time"] == fl_df["time"][ts]]
# pl_df.loc[pl_df["time"] == fl_df["time"][ts]]

#pl_df

boxm.boxm_ds_unstacked.emi.sel(emi_species="NO", time=fl_df.time[2]).sel(level=178.6, method="nearest")

In [ ]:
fig1, ax1 = plt.subplots()
ax1.set_xticks(np.arange(chem_params["lon_bounds"][0], chem_params["lon_bounds"][1], 0.05))
ax1.set_yticks(np.arange(chem_params["lat_bounds"][0], chem_params["lat_bounds"][1], 0.05))


ts = 6

# Plot the heatmap
heatmap_data = boxm.boxm_ds_unstacked.emi.sel(emi_species="NO", time=fl_df.time[ts]).sel(level=178.6, method="nearest").transpose("latitude", "longitude")
heatmap_data.plot(ax=ax1, cmap='summer')  # You can choose a colormap of your preference


scat_fl = ax1.scatter(fl_df["longitude"].loc[fl_df["time"] == fl_df["time"][ts]],
                      fl_df["latitude"].loc[fl_df["time"] == fl_df["time"][ts]], 
                      s=5, c="red", label="Flight path")

scat_pl = ax1.scatter(pl_df["longitude"].loc[pl_df["time"] == fl_df["time"][ts]],
                      pl_df["latitude"].loc[pl_df["time"] == fl_df["time"][ts]],
                      s=10E-2 * pl_df["width"].loc[pl_df["time"] == fl_df["time"][ts]], c="blue", label="Plume evolution")

ax1.legend(loc="upper left")
ax1.set_xlim([chem_params["lon_bounds"][0], chem_params["lon_bounds"][1]])
ax1.set_ylim([chem_params["lat_bounds"][0], chem_params["lat_bounds"][1]])
#plt.grid()
plt.show()

In [ ]:
boxm.boxm_ds_unstacked.emi.sel(emi_species="NO", time=fl_df.time[2]).sel(level=178.6, method="nearest").transpose("latitude", "longitude").plot()

In [ ]:
boxm.mc_test(fl_df)

In [13]:
# # Select the DataArray for the specified emi_species
# grid_mass = boxm.boxm_ds_unstacked.emi.sel(emi_species="NO", time="2022-01-20T13:030:00.000000000").sel(level=178.6, method="nearest") #.transpose("latitude", "longitude").plot()

# grid_mass_over_zero = grid_mass.where(grid_mass > 0, drop=True)
# grid_mass_over_zero



In [14]:
# plt.figure(figsize=(10, 6))  # Optional: Adjust figure size

# boxm.boxm_ds["M"].sel(cell=14).plot()
# #boxm.boxm_ds["Y_orig"].sel(species="NO").plot()
# plt.legend()
# plt.show()
#fl_gen.anim_fl(fl_df, pl_df)

In [15]:
# boxm.anim_chem("emi", "NO", 178.6)